IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M5\FAULT_M5_5


VOLO m5 FAULT 5%

In [3]:
rcou_m5_fault5 = pd.read_csv("RCOU.csv")
rcou_m5_fault5 = rcou_m5_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

rcou_m5_fault5 = rcou_m5_fault5[((rcou_m5_fault5['C9'] >= 1400) & (rcou_m5_fault5['C10'] >= 1400) & (rcou_m5_fault5['C11'] >= 1400) & (rcou_m5_fault5['C12'] >= 1400) & (rcou_m5_fault5['C13'] >= 1400) & (rcou_m5_fault5['C14']>= 1400))]

pd.set_option('display.max_rows',700)
display(rcou_m5_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
1769,76753558,1527,1406,1443,1496,1502,1436
1770,76764123,1401,1543,1479,1473,1402,1541
1771,76775256,1414,1543,1476,1488,1405,1550
1772,76785781,1439,1533,1467,1509,1414,1554
1773,76796106,1464,1523,1461,1525,1428,1554
...,...,...,...,...,...,...,...
6607,129946886,1575,1480,1528,1530,1552,1506
6608,129956887,1582,1463,1520,1530,1555,1494
6609,129967010,1590,1438,1512,1525,1563,1471
6610,129977954,1583,1423,1507,1510,1561,1451


In [4]:
xkf1_m5_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m5_fault5 = xkf1_m5_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_fault5 = xkf1_m5_fault5[((xkf1_m5_fault5['TimeUS'] >= 76753558) & (xkf1_m5_fault5['TimeUS'] <= 129991804))]
print(xkf1_m5_fault5)

         TimeUS  Roll  Pitch    Yaw
1772   76762027  0.58  -0.33  17.99
1773   76773040  0.56  -0.33  17.99
1774   76783338  0.60  -0.33  18.00
1775   76794016  0.62  -0.31  18.00
1776   76805524  0.72  -0.37  17.99
...         ...   ...    ...    ...
6610  129944934 -0.74  -2.38   1.18
6611  129954953 -0.69  -2.42   1.17
6612  129964991 -0.62  -2.45   1.17
6613  129975220 -0.57  -2.47   1.15
6614  129989898 -0.34  -2.45   1.13

[4843 rows x 4 columns]


In [5]:
att_m5_fault5 = pd.read_csv("ATT.csv")
att_m5_fault5 = att_m5_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_fault5 = att_m5_fault5[((att_m5_fault5['TimeUS'] >= 76753558) & (att_m5_fault5['TimeUS'] <= 129991804))]
print(att_m5_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
1773   76763431     0.05  0.58     -0.03  -0.33   17.98  17.99
1774   76774288     0.05  0.56     -0.02  -0.33   17.98  17.99
1775   76784991     0.05  0.60     -0.01  -0.33   17.98  18.00
1776   76795289     0.04  0.62      0.00  -0.31   17.98  18.00
1777   76807108     0.04  0.72      0.00  -0.37   17.98  17.99
...         ...      ...   ...       ...    ...     ...    ...
6611  129945990     0.51 -0.74     -0.50  -2.38    5.87   1.18
6612  129956090     0.54 -0.69     -0.43  -2.42    5.87   1.17
6613  129966271     0.56 -0.62     -0.37  -2.45    5.87   1.17
6614  129977309     0.56 -0.57     -0.32  -2.47    5.87   1.15
6615  129991025     0.56 -0.34     -0.28  -2.45    5.87   1.13

[4843 rows x 7 columns]


In [6]:
att_m5_fault5 = pd.merge_asof(att_m5_fault5,xkf1_m5_fault5, on="TimeUS", by=["Roll","Pitch","Yaw"], tolerance=100000)
print(att_m5_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      76763431     0.05  0.58     -0.03  -0.33   17.98  17.99
1      76774288     0.05  0.56     -0.02  -0.33   17.98  17.99
2      76784991     0.05  0.60     -0.01  -0.33   17.98  18.00
3      76795289     0.04  0.62      0.00  -0.31   17.98  18.00
4      76807108     0.04  0.72      0.00  -0.37   17.98  17.99
...         ...      ...   ...       ...    ...     ...    ...
4838  129945990     0.51 -0.74     -0.50  -2.38    5.87   1.18
4839  129956090     0.54 -0.69     -0.43  -2.42    5.87   1.17
4840  129966271     0.56 -0.62     -0.37  -2.45    5.87   1.17
4841  129977309     0.56 -0.57     -0.32  -2.47    5.87   1.15
4842  129991025     0.56 -0.34     -0.28  -2.45    5.87   1.13

[4843 rows x 7 columns]


In [7]:
esc_0_m5_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m5_fault5 = esc_0_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_fault5 = esc_0_m5_fault5[((esc_0_m5_fault5['TimeUS'] >= 76753558) & (esc_0_m5_fault5['TimeUS'] <= 129991804))]
print(esc_0_m5_fault5)

        TimeUS   RPM  Curr
167   76777380  4018  3.31
168   76884292  4491  3.44
169   76987961  5211  4.30
170   77102140  5761  4.59
171   77318374  6075  4.93
172   77419594  6455  5.00
173   77538251  6402  5.53
174   77646886  6668  5.07
175   77752762  6626  5.65
176   77863344  6585  5.39
177   77970283  6591  5.12
178   78083434  6708  5.13
179   78189598  6586  5.08
180   78304571  6651  4.47
181   78419002  6362  4.20
182   78519971  6287  4.21
183   78627943  6317  4.22
184   78842090  6171  4.53
185   78960916  6257  5.16
186   79069112  6273  4.65
187   79168267  6442  4.75
188   79389719  6444  4.59
189   79496014  6518  5.05
190   79605347  6605  4.18
191   79714896  6394  4.48
192   79819400  6277  4.27
193   79926271  6099  4.15
194   80045948  6071  4.01
195   80148113  6065  4.40
196   80261730  6003  4.34
197   80368502  6197  3.92
198   80474807  6159  3.50
199   80576537  5997  3.19
200   80690312  5843  4.14
201   80807273  5877  3.98
202   80912583  6025  4.07
2

In [8]:
imu_2_m5_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m5_fault5 = imu_2_m5_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_fault5 = imu_2_m5_fault5[((imu_2_m5_fault5['TimeUS'] >= 76753558) & (imu_2_m5_fault5['TimeUS'] <= 129991804))]
print(imu_2_m5_fault5)

          TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ
7016    76755915  0.221122 -0.091488 -0.037913 -0.018269 -0.139721  -9.904168
7017    76757791 -0.117904 -0.007784  0.008021 -0.038367 -0.026568 -10.149410
7018    76760653 -0.280996  0.063489  0.042730 -0.018728  0.092308 -10.194450
7019    76763257 -0.136426  0.033357  0.039310  0.064935  0.159984  -9.827131
7020    76766913  0.194567 -0.092251 -0.007223  0.070961  0.174333  -9.837384
...          ...       ...       ...       ...       ...       ...        ...
25402  129976967  0.187293 -0.040562 -0.023928 -0.650505  0.839263 -11.969900
25403  129979707  0.243907  0.011482 -0.016532 -0.572262  0.844849 -11.934100
25404  129982706  0.288090 -0.037832 -0.023002 -0.592323  0.822716 -12.020670
25405  129985022  0.250959 -0.037079 -0.017797 -0.633674  0.866556 -12.446930
25406  129990913  0.320440 -0.026997 -0.025026 -0.625666  0.886532 -13.139530

[18391 rows x 7 columns]


In [9]:
m5_fault5 = pd.merge_asof(att_m5_fault5, imu_2_m5_fault5, on="TimeUS",  tolerance= 100000)
print(m5_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw      GyrX  \
0      76763431     0.05  0.58     -0.03  -0.33   17.98  17.99 -0.136426   
1      76774288     0.05  0.56     -0.02  -0.33   17.98  17.99 -0.067289   
2      76784991     0.05  0.60     -0.01  -0.33   17.98  18.00 -0.030828   
3      76795289     0.04  0.62      0.00  -0.31   17.98  18.00  0.021858   
4      76807108     0.04  0.72      0.00  -0.37   17.98  17.99  0.113096   
...         ...      ...   ...       ...    ...     ...    ...       ...   
4838  129945990     0.51 -0.74     -0.50  -2.38    5.87   1.18  0.126391   
4839  129956090     0.54 -0.69     -0.43  -2.42    5.87   1.17  0.125664   
4840  129966271     0.56 -0.62     -0.37  -2.45    5.87   1.17  0.159165   
4841  129977309     0.56 -0.57     -0.32  -2.47    5.87   1.15  0.187293   
4842  129991025     0.56 -0.34     -0.28  -2.45    5.87   1.13  0.320440   

          GyrY      GyrZ      AccX      AccY       AccZ  
0     0.033357  0.039310  0.0

In [10]:
m5_fault5 = pd.merge_asof(esc_0_m5_fault5, m5_fault5, on="TimeUS",  tolerance= 100000)
print(m5_fault5)

        TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw  \
0     76777380  4018  3.31     0.05   0.56     -0.02  -0.33   17.98   17.99   
1     76884292  4491  3.44     0.05   0.75      0.14  -0.76   17.98   18.06   
2     76987961  5211  4.30     0.01   0.75      0.39  -0.84   17.98   18.47   
3     77102140  5761  4.59    -0.01   0.55      1.07  -0.89   17.98   19.42   
4     77318374  6075  4.93    -0.59   1.01      2.89   0.48   17.97   21.49   
5     77419594  6455  5.00    -0.93   0.61      3.76   1.41   17.96   22.43   
6     77538251  6402  5.53    -1.69   0.57      4.38   2.69   17.49   23.06   
7     77646886  6668  5.07    -2.40   0.33      4.58   3.66   15.62   23.40   
8     77752762  6626  5.65    -2.97  -0.40      4.65   3.52   13.73   23.10   
9     77863344  6585  5.39    -3.45  -1.11      4.74   3.29   11.85   22.10   
10    77970283  6591  5.12    -3.75  -1.75      5.02   3.63    9.73   20.66   
11    78083434  6708  5.13    -3.90  -2.40      5.35

In [11]:
m5_fault5["TimeUS"] = m5_fault5["TimeUS"] - m5_fault5.iloc[0]["TimeUS"]
m5_fault5["TimeUS"] = m5_fault5["TimeUS"].astype(int)
m5_fault5["Guasto"] = 1
print(m5_fault5)

       TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw  \
0           0  4018  3.31     0.05   0.56     -0.02  -0.33   17.98   17.99   
1      106912  4491  3.44     0.05   0.75      0.14  -0.76   17.98   18.06   
2      210581  5211  4.30     0.01   0.75      0.39  -0.84   17.98   18.47   
3      324760  5761  4.59    -0.01   0.55      1.07  -0.89   17.98   19.42   
4      540994  6075  4.93    -0.59   1.01      2.89   0.48   17.97   21.49   
5      642214  6455  5.00    -0.93   0.61      3.76   1.41   17.96   22.43   
6      760871  6402  5.53    -1.69   0.57      4.38   2.69   17.49   23.06   
7      869506  6668  5.07    -2.40   0.33      4.58   3.66   15.62   23.40   
8      975382  6626  5.65    -2.97  -0.40      4.65   3.52   13.73   23.10   
9     1085964  6585  5.39    -3.45  -1.11      4.74   3.29   11.85   22.10   
10    1192903  6591  5.12    -3.75  -1.75      5.02   3.63    9.73   20.66   
11    1306054  6708  5.13    -3.90  -2.40      5.35   4.23    7.

In [12]:
path_file = path_file.replace(r"m5\FAULT_m5_5", "")
os.chdir(path_file)
m5_fault5.to_csv('m5_fault5.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M5\FAULT_M5_5
